In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, MaxPool2D
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import *

In [15]:
len(tf.config.list_physical_devices('GPU'))

1

In [33]:
def import_maps(path):
    images = []
    for index, name in enumerate(os.listdir(path)):
        folder = os.path.join(path, name)
        for file_class in os.listdir(folder):
            if 'patches' in file_class:
                im_folder = os.path.join(folder, file_class)
#                 print(im_folder)
#                 break
                for im in os.listdir(im_folder):
                    img = cv2.imread(os.path.join(im_folder, im))
                    #print(img.shape)
                    if img is not None:
                        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                        img1 = cv2.resize(img, (64, 64))
                        img = np.dstack((img1,img1,img1))
                    if img is not None:
                   #     img = (img-np.mean(img))/np.std(img)
                        images.append((np.array(img), index)) 
    return images

In [34]:
image_train_set = import_maps(r'C:\Users\AIIMS-IITD\Desktop\Sanjeev\4_class_cluster_volume_seg\Train')
image_test_set = import_maps(r'C:\Users\AIIMS-IITD\Desktop\Sanjeev\4_class_cluster_volume_seg\Test')

In [35]:
len(image_train_set)

7623

In [36]:
len(image_test_set)

1110

In [37]:
train_images_all = [i[0] for i in image_train_set]
train_images_array = np.array(train_images_all)
#train_images_array=np.expand_dims(train_images_array,axis=3)
train_image_label = [i[1] for i in image_train_set]
train_image_label = np.array(train_image_label)

In [38]:
test_images_all = [i[0] for i in image_test_set]
test_images_array = np.array(test_images_all)
#test_images_array=np.expand_dims(test_images_array,axis=3)
test_image_label = [i[1] for i in image_test_set]
test_image_label = np.array(test_image_label)

In [39]:
scalar = StandardScaler(copy=False)
train_images = scalar.fit_transform(train_images_array.reshape(len(train_images_array), 64*64*3))
train_images_array = train_images.reshape(len(train_images_array), 64,64, 3)

In [40]:
test_image = scalar.fit_transform(test_images_array.reshape(len(test_images_array),64*64*3))
test_image_array = test_image.reshape(len(test_images_array),64,64, 3)

In [41]:
enc = OneHotEncoder(handle_unknown='ignore')
train_label_enc = enc.fit_transform(train_image_label.reshape(-1, 1)).toarray()
#val_label_enc = enc.fit_transform(val_y.reshape(-1, 1)).toarray()
test_label_enc = enc.fit_transform(test_image_label.reshape(-1, 1)).toarray()

In [91]:
model = Sequential()
model.add(Conv2D(input_shape=(64,64,3),filters=64,kernel_size=(5,5),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))



model.add(Conv2D(filters=16,kernel_size=(3,3),padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(MaxPooling2D(2,2))
model.add(Dropout(0.5))

model.add(Flatten(name='flatten'))
#model.add(Dense(16, activation='relu', name='fc1'))
model.add(Dense(4, activation='sigmoid'))

model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=keras.optimizers.SGD(0.001),
               metrics=['accuracy'])
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_72 (Conv2D)           (None, 64, 64, 64)        4864      
_________________________________________________________________
batch_normalization_49 (Batc (None, 64, 64, 64)        256       
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 32, 32, 32)        18464     
_________________________________________________________________
batch_normalization_50 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_52 (Dropout)         (None, 16, 16, 32)      

In [ ]:
history = model.fit(train_images_array,train_label_enc,validation_split=0.2, batch_size=32, epochs=500, verbose=1,shuffle=True)

Epoch 1/500
191/191 [==============================] - 2s 8ms/step - loss: 1.5149 - accuracy: 0.4761 - val_loss: 2.1092 - val_accuracy: 0.0020
Epoch 2/500
191/191 [==============================] - 1s 7ms/step - loss: 0.9526 - accuracy: 0.6366 - val_loss: 2.9143 - val_accuracy: 0.0289
Epoch 3/500
191/191 [==============================] - 1s 7ms/step - loss: 0.6992 - accuracy: 0.7343 - val_loss: 3.6683 - val_accuracy: 0.0413
Epoch 4/500
191/191 [==============================] - 1s 7ms/step - loss: 0.5814 - accuracy: 0.7817 - val_loss: 4.0698 - val_accuracy: 0.0485
Epoch 5/500
191/191 [==============================] - 1s 7ms/step - loss: 0.4745 - accuracy: 0.8221 - val_loss: 4.2515 - val_accuracy: 0.0505
Epoch 6/500
191/191 [==============================] - 1s 7ms/step - loss: 0.4139 - accuracy: 0.8468 - val_loss: 4.1348 - val_accuracy: 0.0557
Epoch 7/500
191/191 [==============================] - 1s 7ms/step - loss: 0.3362 - accuracy: 0.8785 - val_loss: 4.4745 - val_accuracy: 0.0479

In [90]:
score = model.evaluate(test_image_array,test_label_enc, batch_size=32)

35/35 [==============================] - 0s 3ms/step - loss: 2.2006 - accuracy: 0.2964


# Old

In [29]:
model = Sequential()

model.add(Conv2D(input_shape=(112,112,3),filters=128,kernel_size=(3,3),padding="same", activation='LeakyReLU'))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="LeakyReLU"))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Dropout(0.2))

model.add(Conv2D(filters=48,kernel_size=(1,1),padding="same", activation="LeakyReLU"))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32,kernel_size=(1,1),padding="same", activation="LeakyReLU"))
model.add(Dropout(0.2))

model.add(Conv2D(filters=24,kernel_size=(1,1),padding="same", activation="LeakyReLU"))
model.add(Dropout(0.2))

model.add(Flatten(name='flatten'))
model.add(Dense(16, activation='LeakyReLU', name='fc1'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='sigmoid', name='fc2'))

model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=keras.optimizers.Adam(0.0012),
               metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 112, 112, 128)     3584      
_________________________________________________________________
dropout_12 (Dropout)         (None, 112, 112, 128)     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 112, 112, 64)      73792     
_________________________________________________________________
dropout_13 (Dropout)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 112, 112, 64)      36928     
_________________________________________________________________
dropout_14 (Dropout)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 48)     

In [30]:
from tensorflow.keras.layers import LeakyReLU


In [31]:
history = model.fit(train_images_array,train_label_enc,validation_data=(test_images_array,test_label_enc), batch_size=32, epochs=5, verbose=1,shuffle=True)

Epoch 1/5
239/239 [==============================] - 19s 77ms/step - loss: 0.4049 - accuracy: 0.8607 - val_loss: 328.5982 - val_accuracy: 0.3108
Epoch 2/5
239/239 [==============================] - 18s 77ms/step - loss: 0.0630 - accuracy: 0.9820 - val_loss: 359.2047 - val_accuracy: 0.3378
Epoch 3/5
239/239 [==============================] - 18s 77ms/step - loss: 0.0249 - accuracy: 0.9923 - val_loss: 403.9989 - val_accuracy: 0.3099
Epoch 4/5
239/239 [==============================] - 18s 77ms/step - loss: 0.0528 - accuracy: 0.9866 - val_loss: 279.6661 - val_accuracy: 0.3432
Epoch 5/5
239/239 [==============================] - 18s 77ms/step - loss: 0.0682 - accuracy: 0.9833 - val_loss: 336.4004 - val_accuracy: 0.2108


In [32]:
score = model.evaluate(test_image_array,test_label_enc, batch_size=16)

70/70 [==============================] - 1s 11ms/step - loss: 9.9756 - accuracy: 0.1937
